## Training SNNs on temporal sequence MNIST data using Friedemann Zenke's network formulation (spytorch)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import torch
from pathlib import Path
from model_spytorch import SNNModel

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-u_81fl0s because the default path (/home/jupyter-ikharitonov/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


ModuleNotFoundError: No module named 'torch'

In [ ]:
data = np.load(Path.home()/'RANCZLAB-NAS/iakov/data/mnist_sequences.npy')

In [ ]:
data = data.reshape((data.shape[0], data.shape[1], data.shape[2]*data.shape[3]))
data = torch.Tensor(data)

In [ ]:
# preprocessing
data = data[torch.randperm(data.shape[0])] # row shuffling
data /= 255 # 0-1 normalisation
data *= 0.5 # scaling down by 1/2

In [ ]:
data.shape

In [ ]:
model = SNNModel(batch_size = 100, hidden_units = data.shape[2], num_timesteps = data.shape[1], step_length = 1/data.shape[1], device = torch.device('cuda'), dtype = torch.float)

In [ ]:
model.init_parameters()

In [ ]:
save_weights_path = Path.home() / 'RANCZLAB-NAS/iakov/v1_weights_31_jan'

In [ ]:
losses = model.train(data, lr = 0.1, num_epochs = 100, save_weights_path = save_weights_path)

In [ ]:
np.save(save_weights_path/'losses.npy', losses)

In [ ]:
plt.figure()
plt.plot(losses)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(model.v1.detach().cpu().numpy())

In [ ]:
# TEST
mem_rec, spk_rec = model.run_snn(data[:100].cuda())

In [ ]:
mem_rec.shape

In [ ]:
plt.imshow(spk_rec.reshape((100,210,28,28)).detach().cpu().numpy()[0,52,:,:])

In [ ]:
# Animation takes long time to load
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig, ax = plt.subplots(nrows=1,ncols=3)

def animate(t):
    plt.cla()
    ax[0].set_title(f'Frame {t}: input', fontsize=5)
    ax[0].imshow(data[:100,:,:].reshape((100,210,28,28)).detach().cpu().numpy()[0,t,:,:])
    ax[1].set_title(f'Frame {t}: membrane voltage', fontsize=5)
    ax[1].imshow(mem_rec.reshape((100,210,28,28)).detach().cpu().numpy()[0,t,:,:])
    ax[2].set_title(f'Frame {t}: unit spiking', fontsize=5)
    ax[2].imshow(spk_rec.reshape((100,210,28,28)).detach().cpu().numpy()[0,t,:,:])

matplotlib.animation.FuncAnimation(fig, animate, frames=210)